In [1]:
from astropy.coordinates.angle_utilities import angular_separation
from astropy.coordinates import SkyCoord, get_sun,FK4
from astropy.time import Time

from datetime import timedelta
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.dates as mdates

import os
import glob
import numpy as np
import astropy.units as u

from collections import Counter

In [2]:
def bandToFreq(band):
    # Convert the input to a numpy array (if it's not already)
    band = np.asarray(band)
    
    # Create a mapping of band numbers to frequency values
    band_to_freq = {
        1: 0.45, 2: 0.70, 3: 0.90, 4: 1.31,
        5: 2.20, 6: 3.93, 7: 4.70, 8: 6.55, 9: 9.18
    }

    # Use np.vectorize to apply the mapping to each element in the array
    freq = np.vectorize(lambda b: band_to_freq.get(b, -1))(band)
    
    return freq

In [3]:
def sigmaClip(y, N):
    y = np.asarray(y)
    overall_mask = np.ones(len(y), dtype=bool)  # Initialize overall mask to all True
    
    for _ in range(N):
        median = np.median(y[overall_mask])  # Calculate median only on filtered values
        std_dev = np.std(y[overall_mask])    # Calculate std_dev only on filtered values
        
        # Mask values within 1 sigma of the mean
        current_mask = (y >= median - std_dev) & (y <= median + std_dev)
        overall_mask &= current_mask  # Update overall mask with new filtering
    
    return overall_mask

In [4]:
# Load the file to inspect its contents
filename = r"C:\Users\adamf\Documents\PhD\Diffraction\interpolatedRAE2MasterFile.csv"

rawData = pd.read_csv(filename)
data = rawData
data['time'] = pd.to_datetime(data['time'])
data.drop('Unnamed: 0',axis=1,inplace=True)
data.set_index('time',inplace=True)

In [5]:
data.iloc[0]

frequency_band         1.000000e+00
position_x             2.545710e+03
position_y             7.625042e+02
position_z            -8.931800e+02
earth_unit_vector_x    1.177206e-01
earth_unit_vector_y    9.016418e-01
earth_unit_vector_z    4.161535e-01
right_ascension        8.605652e-01
declination           -1.015174e+01
rv1_coarse             4.044755e+06
rv2_coarse             2.469941e+08
rv1_fine               0.000000e+00
rv2_fine               0.000000e+00
rv_temp                1.092344e+09
Name: 1973-07-12 00:00:12.304000, dtype: float64

In [6]:
start_date = pd.to_datetime("1973-11-01 7:00")
end_date = pd.to_datetime("1973-11-01 9:00")

# Selecting rows within the date range
earthOccult = data[(data.index >= start_date) & (data.index <= end_date)].copy()

In [7]:
earthOccult.head(50)

,frequency_band,position_x,position_y,position_z,earth_unit_vector_x,earth_unit_vector_y,earth_unit_vector_z,right_ascension,declination,rv1_coarse,rv2_coarse,rv1_fine,rv2_fine,rv_temp
time,,,,,,,,,,,,,,
1973-11-01 07:00:00.222,7,1683.539185,1371.421509,-1789.884827,-0.295881,0.886780,0.355071,2.090268,-32.820148,3.545601e+05,4.804991e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:02.147,7,1681.701599,1373.111023,-1790.289398,-0.295886,0.886779,0.355070,2.094167,-32.835135,3.698107e+05,4.338489e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:04.072,7,1679.864014,1374.800537,-1790.693970,-0.295890,0.886778,0.355069,2.098065,-32.850121,3.545601e+05,4.804991e+05,2.844006e+05,4.079849e+05,1092343562
1973-11-01 07:00:05.997,7,1678.026428,1376.490051,-1791.098541,-0.295895,0.886777,0.355068,2.101963,-32.865107,3.545601e+05,4.338489e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:07.922,7,1676.188843,1378.179565,-1791.503113,-0.295899,0.886776,0.355067,2.105861,-32.880093,3.254992e+05,5.321649e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:09.847,7,1674.351257,1379.869080,-1791.907684,-0.295903,0.886775,0.355066,2.109760,-32.895079,3.698107e+05,3.917275e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:13.657,6,1670.666046,1383.239899,-1792.706085,-0.295912,0.886772,0.355064,2.117567,-32.924824,5.533608e+05,4.378609e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:15.582,6,1668.818420,1384.921204,-1793.099915,-0.295916,0.886771,0.355064,2.121476,-32.939583,5.533608e+05,4.378609e+05,0.000000e+00,0.000000e+00,1092343562
1973-11-01 07:00:17.507,6,1666.970795,1386.602509,-1793.493744,-0.295921,0.886770,0.355063,2.125385,-32.954342,5.533608e+05,4.378609e+05,0.000000e+00,0.000000e+00,1092343562


In [8]:
frequencies = np.sort(earthOccult['frequency_band'].unique())

In [9]:
earthOccult['TimeSince'] = (earthOccult.index-earthOccult.index[0]).total_seconds()

In [36]:
first_date = earthOccult.index[0]
date_title = first_date.strftime("%d %B")

# Plot with the overall title
%matplotlib qt
fig, axes = plt.subplots(3, 3, figsize=(9, 5 * len(frequencies)), sharex=True)

# Add the overall title with the day and month
fig.suptitle(f"Sun Occultation Observations - {date_title}", fontsize=16)

# Determine the number of columns in the axes grid
num_cols = axes.shape[1]

for i, (ax, freq) in enumerate(zip(axes.flatten(), frequencies)):
    # Subset for the current frequency band
    subset = earthOccult[earthOccult['frequency_band'] == freq]
    x = subset.index  # Use the index as the x-axis
    
    mask = sigmaClip(subset['rv1_coarse'], 5)
    ax.scatter(x[~mask], subset['rv1_coarse'][~mask])
    ax.set_yscale('log')
    ax.set_title(f'Frequency: {bandToFreq(freq)} MHz')
    ax.grid(True)

    # Format x-axis to show time (every 30 minutes)
    if i // num_cols == axes.shape[0] - 1:  # Only configure x-axis for the last row
        ax.set_xlabel('Time')
        ax.xaxis.set_major_locator(mdates.MinuteLocator(byminute=[30], interval=1))  # 30-minute intervals
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))   # Format as HH:MM

    # Only show y-axis labels for subplots in the leftmost column
    if i % num_cols == 0:  # Check if the subplot is in the first column
        ax.set_ylabel('Upper V (Uncalibrated)')
        ax.yaxis.set_label_position("left")  # Ensure the label is on the left
        ax.yaxis.tick_left()  # Ensure ticks are on the left side

# Adjust spacing to make room for the title
fig.subplots_adjust(top=0.92, hspace=0.4, wspace=0.4)

plt.show()

In [50]:
%matplotlib qt
freq = 1

subset = earthOccult[earthOccult['frequency_band']==freq]
x = subset.index
    
    
plt.suptitle('Lunar Occultation of the Earth',fontsize = 16) 
mask = sigmaClip(subset['rv1_coarse'],2)
plt.scatter(x,subset['rv1_coarse'])

plt.yscale('log')
plt.title(f'Frequency: {bandToFreq(freq)}MHz')
plt.xlabel('Time')
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(byminute=[30], interval=1))  # 30-minute intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.ylabel('Upper V (Uncalibrated)')
plt.grid(True)

In [22]:
%matplotlib qt

# Define offset factor to separate plots visually
offset_factor = 1.1
offset = np.arange(len(earthOccult['frequency_band'].unique())) * offset_factor  # Prevent overlap

# Convert 'TimeSince' to manageable units if it's in seconds
earthOccult['TimeSinceHours'] = earthOccult['TimeSince'] / 3600  # Convert to hours (if in seconds)

# Plot each normalized curve with an offset
plt.figure(figsize=(10, 6))
for i, freq in enumerate(np.sort(earthOccult['frequency_band'].unique())):
    # Extract the subset for the current frequency
    subset = earthOccult[earthOccult['frequency_band'] == freq]
    x = subset.index  # Use the converted hours scale

    # Normalize the 'rv1_coarse' for this frequency subset
    rv1_coarse = subset['rv1_coarse'].values  # Convert to numpy array for normalization
    normalized_rv1 = (rv1_coarse - rv1_coarse.min()) / (rv1_coarse.max() - rv1_coarse.min())

    # Add offset and plot
    plt.scatter(x, normalized_rv1 + offset[i], label=f'{bandToFreq(freq)} MHz')

# Set the y-axis to log scale if needed (uncomment below)
# plt.yscale('log')

# Title and labels
plt.title('Type III Solar Burst- 01/11/73')
plt.xlabel('Time')
plt.ylabel('Upper V (Uncalibrated)')

# Format x-axis with fewer ticks
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(byminute=[0,30], interval=1))  # 1-hour intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))  # Format as HH:MM

# Add legend and grid
plt.legend()
plt.grid(True)

plt.gca().yaxis.set_ticks([])

# Show plot
plt.show()